In [7]:
import numpy as np 
import pandas as pd 

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import ElasticNetCV

seed = 88

import os 

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from vecstack import stacking

from sklearn.metrics import r2_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor
import xgboost as xgb

from sklearn.ensemble import AdaBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Binarizer, StandardScaler

In [4]:
INPUT = '/media/tin/DATA/Kaggle/Mercedes/Input/'
OUTPUT = '/media/tin/DATA/Kaggle/Mercedes/Output/'

train = pd.read_csv(os.path.join(INPUT, 'train_engineer.csv'))
test = pd.read_csv(os.path.join(INPUT, 'test_engineer.csv'))

y_train = train["y"]
x_train = train.drop('y', axis=1)

test_id = pd.read_csv(os.path.join(INPUT, 'test.csv'))["ID"]


print('Shape train: {}\nShape test: {}\nShape y_train: {}\nShape x_train: {}'
      .format(train.shape,
              test.shape, 
              y_train.shape, 
              x_train.shape))

#next step probably is to pipeline a system of hyperparameter tuning

Shape train: (4209, 653)
Shape test: (4209, 652)
Shape y_train: (4209,)
Shape x_train: (4209, 652)


In [5]:
################
#use cross validation to estimate performance, not create validation set
################
################

#prepare keras for stack
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.metrics import r2_score, mean_squared_error
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def keras_model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(.6))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.add(Dropout(.6))
         
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.add(Dropout(.6))
    
    model.add(Dense(input_dims//4, activation='sigmoid'))
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    #Visualize NN architecture
    #print(model.summary())
    return model

input_dims = x_train.shape[1]  



In [8]:
models = [
        make_pipeline(StandardScaler(), ElasticNetCV(l1_ratio=0.9500000000000001, tol=1e-05)),
       
        make_pipeline(Binarizer(threshold=0.65),
        AdaBoostRegressor(learning_rate=0.001, loss="linear")),
        
        ExtraTreesRegressor(random_state = 0, n_jobs = -1, 
            n_estimators = 100, max_depth = 3),
        
        RandomForestRegressor(random_state = 0, n_jobs = -1, 
           n_estimators = 100, max_depth = 3),
    
        DecisionTreeRegressor(max_depth=3, min_samples_leaf=2, min_samples_split=13),
        
        XGBRegressor(seed = 0, nthread = -1, learning_rate = 0.01, 
            n_estimators = 700, max_depth = 5, subsample = 0.96320),

        GradientBoostingRegressor(),
        KerasRegressor(build_fn = keras_model, epochs=66, batch_size=20, verbose = 0)
]

In [9]:
#different is here, not x_train1 but x_train
S_train2, S_test2 = stacking(models, np.array(x_train), np.array(y_train), np.array(test), 
        regression = True, metric = r2_score, n_folds = 5, 
        shuffle = True, random_state = 32, verbose = 2)

#can see what distribution of y different folds looklike?

task:   [regression]
metric: [r2_score]

model 0: [Pipeline]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 0: [0.58828060]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 1: [0.59007008]


...

/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 3: [0.65958352]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 4: [0.46720281]
    ----
    MEAN:   [0.56608338]

model 1: [Pipeline]
    fold 0: [0.58703515]
    fold 1: [0.58667387]
    fold 2: [0.54780290]
    fold 3: [0.66038568]
    fold 4: [0.45773666]
    ----
    MEAN:   [0.56245557]

model 2: [ExtraTreesRegressor]
    fold 0: [0.58581423]
    fold 1: [0.57397109]
    fold 2: [0.54388727]
    fold 3: [0.65644603]
    fold 4: [0.46685344]
    ----
    MEAN:   [0.56045660]

model 3: [RandomForestRegressor]
    fold 0: [0.59067705]
    fold 1: [0.58388572]
    fold 2: [0.54421462]
    fold 3: [0.65962076]
    fold 4: [0.45725469]
    ----
    MEAN:   [0.56180804]

model 4: [DecisionTreeRegressor]
    fold 0: [0.58388021]
    fold 1: [0.59192740]
    fold 2: [0.54716266]
    fold 3: [0.65631671]
    fold 4: [0.45935384]
    ----
    MEAN:   [0.56232052]

model 5: [XGBRegressor]
    fold 0: [0.58965368]
    fold 1: [0.58143172]
    fold 2: [0.53481580]
    fold 3: [0.64187885]
    fold 4: [0.45998055]
    ----
    MEAN:   [0.55696628]


InternalError: Blas GEMM launch failed : a.shape=(20, 652), b.shape=(652, 652), m=20, n=652, k=652
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_dense_1_input_0_1/_47, dense_1/kernel/read)]]
	 [[Node: mul_1/_113 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2624_mul_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/media/tin/DATA/newbigtf/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/media/tin/DATA/newbigtf/lib/python3.5/site-package

In [11]:
S_train3, S_test3 = stacking([make_pipeline(StandardScaler(), 
                                            ElasticNetCV(l1_ratio=0.9500000000000001, tol=1e-05))], 
                             np.array(x_train), np.array(y_train), np.array(test), 
        regression = True, metric = r2_score, n_folds = 5, 
        shuffle = True, random_state = 32, verbose = 2)

task:   [regression]
metric: [r2_score]

model 0: [Pipeline]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 0: [0.58828060]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 1: [0.59007008]


...

    fold 2: [0.55131801]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 3: [0.65958352]


/media/tin/DATA/newbigtf/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


    fold 4: [0.46720281]
    ----
    MEAN:   [0.56608338]



In [12]:
#concatenate new to old
S_train = pd.read_csv(os.path.join(OUTPUT,'1stlevel_train.csv'))
S_test = pd.read_csv(os.path.join(OUTPUT,'1stlevel_test.csv'))

S_train2 = pd.DataFrame(np.concatenate((S_train, S_train3), axis = 1))
S_test2 = pd.DataFrame(np.concatenate((S_test, S_test3), axis = 1))
print(S_train.head())
print(S_train2.head())

           0          1          2          3          4          5  \
0  94.014634  94.028091  94.250107  94.028091  99.689148  95.651587   
1  94.203353  94.277061  94.297636  94.274731  93.529366  94.112695   
2  77.884551  78.029300  77.863531  77.964129  78.036446  78.969166   
3  77.836124  77.867430  77.711647  77.794604  78.446503  78.498305   
4  77.540884  77.912067  78.297408  77.912067  78.425507  81.148355   

            6  
0  101.608490  
1   93.318398  
2   75.926529  
3   74.671066  
4   78.916313  
           0          1          2          3          4          5  \
0  94.014634  94.028091  94.250107  94.028091  99.689148  95.651587   
1  94.203353  94.277061  94.297636  94.274731  93.529366  94.112695   
2  77.884551  78.029300  77.863531  77.964129  78.036446  78.969166   
3  77.836124  77.867430  77.711647  77.794604  78.446503  78.498305   
4  77.540884  77.912067  78.297408  77.912067  78.425507  81.148355   

            6          7  
0  101.608490  97.74940

In [13]:
#save to feed to 2nd level
S_train2 = pd.DataFrame(S_train2)
S_test2 = pd.DataFrame(S_test2)

S_train2.to_csv(os.path.join(OUTPUT,'1stlevel_train.csv'), index = False)
S_test2.to_csv(os.path.join(OUTPUT,'1stlevel_test.csv'), index = False)
